In this notebook, I'm going to attempt to run all the techniques on the same circuit

In [1]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits, ghz_circuits
from mitiq import MeasurementResult, Observable, PauliString

In [2]:
n_qubits = 2

In [3]:
circuit = generate_rb_circuits(2, 10)[0]

In [4]:
def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))

    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

In [8]:
from mitiq import zne, ddd, qse, rem, cdr
from functools import partial

qem_executors = {
    "ZNE": zne.mitigate_executor,
    "DDD": ddd.mitigate_executor,
    "QSE": qse.mitigate_executor,
    "REM": rem.mitigate_executor,
    "CDR": cdr.mitigate_executor
}

def compose(executable, qem_method):
    """recursive function that takes in an executable + technique to compose and returns the composed executable"""

    # TODO: further modularize so that just doing a dictionary lookup should allow you to inject the correct args

    match qem_method:
        case "ZNE":
            print('composing ZNE')

            scale_method = zne.scaling.folding.fold_global

            return qem_executors[qem_method](
                executable,
                scale_noise=scale_method
            )
        case "DDD":
            rule = ddd.rules.xx
            return qem_executors[qem_method](
                executable,
                observable=None, # SUS
                rule=rule
            )
        case "QSE":
            Ms = [ # chat GPT generated stabilizer group
                "II",
                "XI",
                "ZI",
                "IX",
                "IZ"
            ]
            check_operators = [
                PauliString(M, coeff=1, support=range(n_qubits)) for M in Ms
            ] # low key copy pasta-ed these so they may not even be the right choice
            negative_Ms_as_pauliStrings = [
                PauliString(M, coeff=-1, support=range(n_qubits)) for M in Ms
            ] # low key copy pasta-ed these so they may not even be the right choice
            code_hamiltonian = Observable(*negative_Ms_as_pauliStrings)

            return qem_executors[qem_method](
                executable,
                check_operators, # not sure what these are, need further reading
                code_hamiltonian,
                observable=None
            )
        case "REM":
            print('composing REM')

            p0 = p1 = 0.05
            icm = rem.generate_inverse_confusion_matrix(2, p0, p1)
            return qem_executors[qem_method](execute, inverse_confusion_matrix=icm)
        case "CDR":
            simulator = partial(execute, noise_level=0, p0=0) # SUS
            return qem_executors[qem_method](
                executable,
                observable=None,
                simulator=simulator
            )
        case _:
            raise ValueError
        

In [14]:
chain = ["QSE", "ZNE"] # MODIFY THIS LINE

executable = execute

for method in chain:
    executable = compose(executable, method)

executable(circuit)

composing ZNE


ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None